In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)
n = 6000  # Nombre d'utilisateurs

# Données utilisateur simulées
df = pd.DataFrame({
    "id": range(1, n+1),
    "age": np.random.randint(18, 70, n),
    "revenu": np.random.randint(15000, 120000, n),
    "type_utilisateur": np.random.choice(["particulier", "pro", "revendeur"], n, p=[0.7, 0.2, 0.1]),
    "nb_annonces_publiées": np.random.poisson(3, n),
    "nb_vehicules_vendus": np.random.poisson(2, n),
    "nb_messages_envoyés": np.random.poisson(10, n),
    "nb_favoris": np.random.poisson(5, n),
    "dernière_connexion": np.random.randint(0, 150, n),  # jours
    "avis": np.round(np.random.uniform(1, 5, n), 1),
    "support_contacté": np.random.randint(0, 5, n)
})

# Simuler des churns selon la logique métier
df["churn"] = 0
for i in df.index:
    if (
        df.loc[i, "dernière_connexion"] > 90 or
        df.loc[i, "avis"] < 2.5 or
        df.loc[i, "support_contacté"] > 3 or
        df.loc[i, "nb_messages_envoyés"] < 2
    ):
        df.loc[i, "churn"] = 1

# Ajouter des utilisateurs au comportement anormal (fraude potentielle)
anomalous_users = np.random.choice(df.index, size=80, replace=False)
df.loc[anomalous_users, "nb_annonces_publiées"] = np.random.randint(50, 300, size=80)
df.loc[anomalous_users, "nb_messages_envoyés"] = np.random.randint(100, 500, size=80)
df.loc[anomalous_users, "avis"] = 1.0

df.head()


,id,age,revenu,type_utilisateur,nb_annonces_publiées,nb_vehicules_vendus,nb_messages_envoyés,nb_favoris,dernière_connexion,avis,support_contacté,churn
0,1,56,58919,particulier,3,3,14,3,112,5.0,4,1
1,2,69,97103,particulier,4,1,5,2,27,3.9,4,1
2,3,46,18358,particulier,2,2,5,6,142,1.9,4,1
3,4,32,103535,particulier,5,2,13,2,111,3.9,1,1
4,5,60,91524,pro,5,3,11,1,76,2.4,2,1


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Préparation
X = df.drop(columns=["id", "churn"])
y = df["churn"]

# Encodage des variables catégorielles
X["type_utilisateur"] = LabelEncoder().fit_transform(X["type_utilisateur"])

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modèle
model = RandomForestClassifier(n_estimators=200, class_weight="balanced", random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Résultats
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.99      1.00       355
           1       1.00      1.00      1.00       845

    accuracy                           1.00      1200
   macro avg       1.00      1.00      1.00      1200
weighted avg       1.00      1.00      1.00      1200



In [3]:
from sklearn.ensemble import IsolationForest

# Isolation Forest pour détecter les utilisateurs "anormaux"
iso = IsolationForest(contamination=0.02, random_state=42)
anomaly_scores = iso.fit_predict(X)

df["anomalie"] = (anomaly_scores == -1).astype(int)

# Voir les utilisateurs suspects
df[df["anomalie"] == 1].head()


,id,age,revenu,type_utilisateur,nb_annonces_publiées,nb_vehicules_vendus,nb_messages_envoyés,nb_favoris,dernière_connexion,avis,support_contacté,churn,anomalie
11,12,28,91777,pro,182,0,201,6,122,1.0,0,1,1
89,90,33,115356,pro,208,2,202,4,17,1.0,2,1,1
142,143,31,98948,particulier,56,3,207,4,114,1.0,2,1,1
172,173,30,32369,pro,170,2,302,3,22,1.0,1,0,1
222,223,20,19827,pro,6,3,5,4,0,1.1,0,1,1
